<a href="https://colab.research.google.com/github/lin941006/Time/blob/main/%E5%80%8B%E4%BA%BA%E6%99%82%E9%96%93%E8%A6%8F%E5%8A%83ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio pandas
import gradio as gr
import pandas as pd
from datetime import datetime, timedelta

tasks = []

category_choices = [
    "生活類",
    "工作/學習類",
    "通勤類",
    "休閒娛樂類",
    "社交互動類",
    "運動與健康類",
    "家務/生活管理類",
    "其他"
]

subcategory_choices = {
    "生活類": ["睡覺", "起床", "吃飯", "洗澡", "休息/發呆"],
    "工作/學習類": ["上班", "上課", "寫作業", "自習", "開會", "報告", "考試", "線上課程"],
    "通勤類": ["捷運", "公車", "火車", "走路", "騎車", "開車"],
    "休閒娛樂類": ["玩手機", "看電影", "追劇", "聽音樂", "看書", "逛街", "看展覽"],
    "社交互動類": ["聊天", "打電話", "聚會", "約會", "家庭時間", "同事交流"],
    "運動與健康類": ["健身", "跑步", "散步", "拉筋", "瑜珈", "冥想", "看醫生"],
    "家務/生活管理類": ["打掃", "洗衣服", "曬衣服", "煮飯", "採買食材", "整理房間", "倒垃圾"]
}

time_choices = [f"{str(h).zfill(2)}:{m}" for h in range(24) for m in ["00", "30"]]
duration_choices = [i * 0.5 for i in range(1, 17)]  # 0.5~8 小時

# -------------------------
# 控制細項顯示
# -------------------------
def toggle_subcategory(category, _):
    if category == "其他":
        return gr.update(visible=False, value=None), gr.update(visible=True, value="")
    elif category:
        sub_choices = subcategory_choices.get(category, [])
        return gr.update(choices=sub_choices, visible=True, value=None), gr.update(visible=False, value="")
    else:
        return gr.update(visible=False, value=None), gr.update(visible=False, value="")

# -------------------------
# 新增任務
# -------------------------
def add_task(category, subcategory, other_input, hours):
    if not category or hours is None:
        return pd.DataFrame(tasks), gr.update(value=None), gr.update(visible=False), gr.update(value=""), gr.update(value=None), gr.update(choices=[task["事項"] for task in tasks], value=None)

    task_name = other_input.strip() if category == "其他" else subcategory
    if not task_name:
        return pd.DataFrame(tasks), gr.update(value=None), gr.update(visible=False), gr.update(value=""), gr.update(value=None), gr.update(choices=[task["事項"] for task in tasks], value=None)

    tasks.append({"事項": task_name, "所需時間(小時)": hours})
    delete_choices = [task["事項"] for task in tasks]

    return pd.DataFrame(tasks), gr.update(value=None), gr.update(visible=False), gr.update(value=""), gr.update(value=None), gr.update(choices=delete_choices, value=None)

# -------------------------
# 產生今日行程
# -------------------------
def generate_schedule(start_time):
    if not tasks or not start_time:
        return "請先新增代辦事項並選擇一天開始時間。"
    current_time = datetime.strptime(start_time, "%H:%M")
    text = "📅 今日行程安排：\n\n"
    for task in tasks:
        end = current_time + timedelta(hours=task["所需時間(小時)"])
        text += f"{current_time.strftime('%H:%M')} ～ {end.strftime('%H:%M')}：{task['事項']}\n"
        current_time = end
    text += "\n🎉 祝福你有美好的一天！"
    return text

# -------------------------
# 清除全部
# -------------------------
def clear_all():
    tasks.clear()
    return pd.DataFrame(tasks), gr.update(value=None), gr.update(visible=False), gr.update(value=""), gr.update(value=None), gr.update(choices=[], value=None), gr.update(value=None), ""

# -------------------------
# 刪除指定任務（同時更新行程）
# -------------------------
def delete_task(selected_task, start_time):
    global tasks
    if selected_task:
        tasks = [task for task in tasks if task["事項"] != selected_task]
    delete_choices = [task["事項"] for task in tasks]
    schedule_text = generate_schedule(start_time) if start_time else ""
    return pd.DataFrame(tasks), gr.update(choices=delete_choices, value=None), schedule_text

# -------------------------
# Gradio 介面
# -------------------------
with gr.Blocks(title="個人時間管理工具") as demo:
    gr.Markdown("## 🕒 個人時間管理工具")
    gr.Markdown("依照分類選擇代辦事項，選擇預計花費時間，系統會自動安排行程並在畫面呈現。")

    # 第一排：行程類型、代辦行程、預計時間
    with gr.Row():
        category_input = gr.Dropdown(choices=category_choices, value=None, label="行程類型")
        subcategory_input = gr.Dropdown(choices=[], value=None, label="代辦行程", visible=False)
        other_input = gr.Textbox(label="其他事項", visible=False)
        time_input = gr.Dropdown(choices=duration_choices, value=None, label="預計花費時間（小時）")
    category_input.change(toggle_subcategory, inputs=[category_input, subcategory_input], outputs=[subcategory_input, other_input])

    # 第二排：新增今日代辦事項按鈕
    add_btn = gr.Button("➕ 新增今日代辦事項")

    # 第三排：今日代辦清單
    task_table = gr.Dataframe(label="📋 今日代辦清單", interactive=False)

    # 第四排：開始時間
    start_time = gr.Dropdown(choices=time_choices, value=None, label="開始時間")

    # 第五排：產生今日行程按鈕
    schedule_btn = gr.Button("📅 產生今日行程")

    # 第六排：今日行程安排
    schedule_output = gr.Textbox(lines=12, label="今日行程安排")

    # 第七排：選擇要刪除的行程
    delete_task_dropdown = gr.Dropdown(choices=[], label="選擇要刪除的行程", value=None)

    # 第八排：刪除指定行程按鈕
    delete_task_btn = gr.Button("🗑️ 刪除指定行程")

    # 第九排：清除所有行程
    clear_btn = gr.Button("🗑️ 清除所有行程")

    # ------------------------- 事件綁定 -------------------------
    add_btn.click(
        add_task,
        inputs=[category_input, subcategory_input, other_input, time_input],
        outputs=[task_table, category_input, subcategory_input, other_input, time_input, delete_task_dropdown]
    )

    schedule_btn.click(
        generate_schedule,
        inputs=start_time,
        outputs=schedule_output
    )

    delete_task_btn.click(
        delete_task,
        inputs=[delete_task_dropdown, start_time],
        outputs=[task_table, delete_task_dropdown, schedule_output]
    )

    clear_btn.click(
        clear_all,
        outputs=[task_table, category_input, subcategory_input, other_input, time_input, delete_task_dropdown, start_time, schedule_output]
    )

demo.launch()
